## AUV Modeling

The equations of Motion are written as 
$$\tau= M\dot{v} + C(v)v +  d(v) + b(\Theta)
$$

where
- $M$ is the inertia matrix
- $C(v)v$ are the coriolis effects
- $d(v)$ are the damping effects
- $b(\Theta)$ are the restoring effects due to buoyancy

In [1]:
from sympy import *
import numpy as np

damping_mode = 'quadratic'
inertia_mode = 'full'

# quaternion components
qw,qx,qy,qz = symbols('q_w q_x q_y q_z')
q = Quaternion.from_Matrix([qw, qx, qy, qz])
# rotation matrix
R = q.to_rotation_matrix()*(qw*qw+qx*qx+qy*qy+qz*qz)
display(R)

# linear and angular velocities
v_x, v_y, v_z = symbols('v_x v_y v_z')
w_x, w_y, w_z = symbols('w_x w_y w_z')
v_lin = Matrix([v_x, v_y, v_z])
v_ang = Matrix([w_x, w_y, w_z])
V = Matrix([v_lin, v_ang])

# linear and angular accelerations
u_dot, v_dot, w_dot = symbols('\dot{v_x} \dot{v_y} \dot{v_z}')
p_dot, q_dot, r_dot = symbols('\dot{w_x} \dot{w_y} \dot{w_z}')
v1_dot = Matrix([u_dot, v_dot, w_dot])
v2_dot = Matrix([p_dot, q_dot, r_dot])
V_dot = Matrix([v1_dot,v2_dot])

# Mass
if inertia_mode == 'full':
    M = Matrix(symbols([['M_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))
else:
    M = Matrix(np.diag([symbols('M_%i%i'%(i,i)) for i in range(1,7)]))
display(M)

inertia = M*V_dot


def skew(v):
    return Matrix([[0,-v[2],v[1]],
                   [v[2],0,-v[0]],
                   [-v[1],v[0],0]])

M11 = M[:3,:3]
M12 = M[:3,3:]
M22 = M[3:,3:]
M21 = M[3:,:3]

# Coriolis
C = Matrix([[zeros(3),-skew(M11*v_lin + M12*v_ang)],
            [-skew(M11*v_lin + M12*v_ang),-skew(M21*v_lin + M22*v_ang)]])
coriolis = C*V

# Damping
Dl = Matrix(symbols([['Dl_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))

Du = Matrix(symbols([['Du_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(v_x)
Dv = Matrix(symbols([['Dv_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(v_y)
Dw = Matrix(symbols([['Dw_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(v_z)
Dp = Matrix(symbols([['Dp_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(w_x)
Dq = Matrix(symbols([['Dq_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(w_y)
Dr = Matrix(symbols([['Dr_%i%i'%(j,i) for i in range(1,7)] for j in range(1,7)]))* \
    abs(w_z)



if damping_mode == 'linear':
    damping = Dl*V
elif damping_mode == 'quadratic':
    damping = (Dl+Du+Dv+Dw+Dp+Dq+Dr)*V
    

# restoring
B, W = symbols('B W') # bouyancy and weight
xb,yb,zb = symbols('x_b y_b z_b')
xg,yg,zg = symbols('x_g y_g z_g')
rg = Matrix([0,0,zg]) # center of gravity
rb = Matrix([0,0,0]) # center of buoyancy

fb = R.T * Matrix([0,0,-B])
fg = R.T * Matrix([0,0,W])

b = Matrix([fg + fb, rg.cross(fg) + rb.cross(fb)])


print('Mass*acc:')
display(inertia)
print('Coriolis matrix:')
display(C)
print('Coriolis effect:')
display(coriolis)
print('Damping:')
display(damping)
print('Restoring:')
display(b)

Matrix([
[q_w**2 + q_x**2 - q_y**2 - q_z**2,            -2*q_w*q_z + 2*q_x*q_y,             2*q_w*q_y + 2*q_x*q_z],
[            2*q_w*q_z + 2*q_x*q_y, q_w**2 - q_x**2 + q_y**2 - q_z**2,            -2*q_w*q_x + 2*q_y*q_z],
[           -2*q_w*q_y + 2*q_x*q_z,             2*q_w*q_x + 2*q_y*q_z, q_w**2 - q_x**2 - q_y**2 + q_z**2]])

Matrix([
[M_11, M_12, M_13, M_14, M_15, M_16],
[M_21, M_22, M_23, M_24, M_25, M_26],
[M_31, M_32, M_33, M_34, M_35, M_36],
[M_41, M_42, M_43, M_44, M_45, M_46],
[M_51, M_52, M_53, M_54, M_55, M_56],
[M_61, M_62, M_63, M_64, M_65, M_66]])

Mass*acc:


Matrix([
[M_11*\dot{v_x} + M_12*\dot{v_y} + M_13*\dot{v_z} + M_14*\dot{w_x} + M_15*\dot{w_y} + M_16*\dot{w_z}],
[M_21*\dot{v_x} + M_22*\dot{v_y} + M_23*\dot{v_z} + M_24*\dot{w_x} + M_25*\dot{w_y} + M_26*\dot{w_z}],
[M_31*\dot{v_x} + M_32*\dot{v_y} + M_33*\dot{v_z} + M_34*\dot{w_x} + M_35*\dot{w_y} + M_36*\dot{w_z}],
[M_41*\dot{v_x} + M_42*\dot{v_y} + M_43*\dot{v_z} + M_44*\dot{w_x} + M_45*\dot{w_y} + M_46*\dot{w_z}],
[M_51*\dot{v_x} + M_52*\dot{v_y} + M_53*\dot{v_z} + M_54*\dot{w_x} + M_55*\dot{w_y} + M_56*\dot{w_z}],
[M_61*\dot{v_x} + M_62*\dot{v_y} + M_63*\dot{v_z} + M_64*\dot{w_x} + M_65*\dot{w_y} + M_66*\dot{w_z}]])

Coriolis matrix:


...

Matrix([
[                                                                                                                                                 w_y*(M_31*v_x + M_32*v_y + M_33*v_z + M_34*w_x + M_35*w_y + M_36*w_z) + w_z*(-M_21*v_x - M_22*v_y - M_23*v_z - M_24*w_x - M_25*w_y - M_26*w_z)],
[                                                                                                                                                 w_x*(-M_31*v_x - M_32*v_y - M_33*v_z - M_34*w_x - M_35*w_y - M_36*w_z) + w_z*(M_11*v_x + M_12*v_y + M_13*v_z + M_14*w_x + M_15*w_y + M_16*w_z)],
[                                                                                                                                                 w_x*(M_21*v_x + M_22*v_y + M_23*v_z + M_24*w_x + M_25*w_y + M_26*w_z) + w_y*(-M_11*v_x - M_12*v_y - M_13*v_z - M_14*w_x - M_15*w_y - M_16*w_z)],
[v_y*(M_31*v_x + M_32*v_y + M_33*v_z + M_34*w_x + M_35*w_y + M_36*w_z) + v_z*(-M_21*v_x - M_22*v_y - M_23*v_z - M_24*w

Damping:


Matrix([
[v_x*(Dl_11 + Dp_11*Abs(w_x) + Dq_11*Abs(w_y) + Dr_11*Abs(w_z) + Du_11*Abs(v_x) + Dv_11*Abs(v_y) + Dw_11*Abs(v_z)) + v_y*(Dl_12 + Dp_12*Abs(w_x) + Dq_12*Abs(w_y) + Dr_12*Abs(w_z) + Du_12*Abs(v_x) + Dv_12*Abs(v_y) + Dw_12*Abs(v_z)) + v_z*(Dl_13 + Dp_13*Abs(w_x) + Dq_13*Abs(w_y) + Dr_13*Abs(w_z) + Du_13*Abs(v_x) + Dv_13*Abs(v_y) + Dw_13*Abs(v_z)) + w_x*(Dl_14 + Dp_14*Abs(w_x) + Dq_14*Abs(w_y) + Dr_14*Abs(w_z) + Du_14*Abs(v_x) + Dv_14*Abs(v_y) + Dw_14*Abs(v_z)) + w_y*(Dl_15 + Dp_15*Abs(w_x) + Dq_15*Abs(w_y) + Dr_15*Abs(w_z) + Du_15*Abs(v_x) + Dv_15*Abs(v_y) + Dw_15*Abs(v_z)) + w_z*(Dl_16 + Dp_16*Abs(w_x) + Dq_16*Abs(w_y) + Dr_16*Abs(w_z) + Du_16*Abs(v_x) + Dv_16*Abs(v_y) + Dw_16*Abs(v_z))],
[v_x*(Dl_21 + Dp_21*Abs(w_x) + Dq_21*Abs(w_y) + Dr_21*Abs(w_z) + Du_21*Abs(v_x) + Dv_21*Abs(v_y) + Dw_21*Abs(v_z)) + v_y*(Dl_22 + Dp_22*Abs(w_x) + Dq_22*Abs(w_y) + Dr_22*Abs(w_z) + Du_22*Abs(v_x) + Dv_22*Abs(v_y) + Dw_22*Abs(v_z)) + v_z*(Dl_23 + Dp_23*Abs(w_x) + Dq_23*Abs(w_y) + Dr_23*Abs(w_z)

Restoring:


Matrix([
[                      -B*(-2*q_w*q_y + 2*q_x*q_z) + W*(-2*q_w*q_y + 2*q_x*q_z)],
[                        -B*(2*q_w*q_x + 2*q_y*q_z) + W*(2*q_w*q_x + 2*q_y*q_z)],
[-B*(q_w**2 - q_x**2 - q_y**2 + q_z**2) + W*(q_w**2 - q_x**2 - q_y**2 + q_z**2)],
[                                                -W*z_g*(2*q_w*q_x + 2*q_y*q_z)],
[                                                W*z_g*(-2*q_w*q_y + 2*q_x*q_z)],
[                                                                             0]])

In [2]:
# thruster wrench
force = rot_axis3(Symbol('psi'))*rot_axis2(Symbol('theta'))*Matrix([Symbol('f'),0,0])
r_f = Matrix([Symbol('x_f'),0,0])
torque = r_f.cross(force)
wrench = Matrix([force, torque])
display(wrench)

Matrix([
[     f*cos(psi)*cos(theta)],
[    -f*sin(psi)*cos(theta)],
[              f*sin(theta)],
[                         0],
[         -f*x_f*sin(theta)],
[-f*x_f*sin(psi)*cos(theta)]])

In [3]:
# Quaternion Error
symbols('q_w q_x q_y q_z')
qd = Quaternion.from_Matrix(symbols('q_w^d q_x^d q_y^d q_z^d'))

print('quaternion error represented as another quaternion')
display((qd.mul(q.inverse())*q.norm()**2).simplify())

# scalar error between two quaternions
# according to https://math.stackexchange.com/questions/90081/quaternion-distance
print('scalar distance')
display(1 - (q.to_Matrix().T*qd.to_Matrix())[0]**2)

quaternion error represented as another quaternion


(q_w*q_w^d + q_x*q_x^d + q_y*q_y^d + q_z*q_z^d) + (q_w*q_x^d - q_w^d*q_x + q_y*q_z^d - q_y^d*q_z)*i + (q_w*q_y^d - q_w^d*q_y - q_x*q_z^d + q_x^d*q_z)*j + (q_w*q_z^d - q_w^d*q_z + q_x*q_y^d - q_x^d*q_y)*k

scalar distance


1 - (q_w*q_w^d + q_x*q_x^d + q_y*q_y^d + q_z*q_z^d)**2